# Introduction

Our primary goals in our research and analysis are to boost Regork's sales and increase operational effectiveness. We have selected three problem statements- relating to the optimal package sizes to position in our business, to the best way to issue coupons to consumers in order to maximize sales, and to atttain maximum coupon redemption rate for more profitable marketing.

We worked on the problem statements and analysed the existing ‘CompleteJourney’ datasets relating to transactions, products, and demographics. We conducted the explorartory data analysis on the datasets to address the problem statements and derive meaningful insights from the data sets like Transactions, Demographics and Product.

## Business Problem 1
Is the retail discount distribution in relation to the age groups ideal?Are discounts yielding higher sales?

For this after cleaning the data we have grouped the ages into 4 groups 19-34, 35-44, 45-54 and 55+ and formed a new data frame called "demo_tran". Then filtering the sales data based up on the retail discount and plotting the required graphs for generating insights addressing the business problem.

<b> We found the below insights after peforming the analysis from the graphs: </b>

* We had noticed that revenue generated by issuing discounts to the 45-54 age group was higher compared to other age group
* Discounts of above 5 dollars have produced little sales, so we can scale down on them.
* Compared to other age categories, the 55 and older age group received the largest percentage of discounts.

In [1]:
from completejourney_py import get_data
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import calendar
df = px.data.tips()
import warnings
warnings.filterwarnings('ignore')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\anjan\\anaconda3\\lib\\site-packages\\matplotlib.libs\\.load-order-matplotlib-3.7.0'

In [ ]:
transactions = get_data()['transactions']
products = get_data()['products']
promotions = get_data()['promotions']
demographics = get_data()['demographics']
transactions = get_data()['transactions']
campaigns = get_data()['campaigns']
coupons = get_data()['coupons']
campaign_descriptions = get_data()['campaign_descriptions']
coupon_redemptions = get_data()['coupon_redemptions']


In [ ]:
demographics.head()

In [ ]:
demo_tran = demographics.merge(transactions, on = ['household_id'], how = 'inner')
demo_tran = demo_tran[~demo_tran.isnull().any(axis=1)]
demo_tran.head()

In [ ]:
demo_tran['age'].value_counts()

In [ ]:
conditions = [
    ((demo_tran['age'] == "19-24") | (demo_tran['age'] == "25-34")),
    ((demo_tran['age'] == "35-44")),
    ((demo_tran['age'] == "45-54")),
    ((demo_tran['age'] == "55-64") | (demo_tran['age'] == "65+"))
    ]

# create a list of the values we want to assign for each condition
values = ['19-34', '35-44', '45-54', '55+']

demo_tran['age_group'] = np.select(conditions, values)


In [ ]:
demo_tran['age_group'].value_counts()

In [ ]:
demo_tran1 = demo_tran.copy()
demo_tran1 = demo_tran1[(demo_tran1['retail_disc'] > 0) & (demo_tran1['retail_disc'] < 1) ].groupby('age_group')\
                            .aggregate({'sales_value':'sum', 'retail_disc':'sum'})

demo_tran1['percentage'] = demo_tran1['retail_disc']*100/demo_tran1['sales_value']
demo_tran1 = demo_tran1.reset_index()
demo_tran1.head()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=demo_tran1.age_group, y=demo_tran1.percentage, name="Percent discount used"),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x=demo_tran1['age_group'], y=demo_tran1['sales_value'], name = 'Total Sales'),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=demo_tran1['age_group'], y=demo_tran1['retail_disc'], name = 'Total Discount'),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="Retail Discount (for less than $1) v/s Total Sales Revenue (by Age Group)"
)

fig.update_yaxes(range=[0,40], secondary_y=True)
# Set x-axis title
fig.update_xaxes(title_text="Age Group")

# Set y-axes titles
fig.update_yaxes(title_text="Total Sales Value", secondary_y=False)
fig.update_yaxes(title_text="Total percentage of discount used", secondary_y=True)

fig.show()

## Business Problem 2
Which package sizes generate the highest sales for the top 5 grocery products. To understand which package size to focus while procuring from manufacturer.

In order to generate insights, we first cleaned the data and identified various product categories in Grocery. To determine which package size is generating maximum sales, we then analysed those products by sales along with their package sizes to determine which package size generates the most revenue.

Based on the graph analysis, we discovered the following insights:<br>
i. Drinking water and soft drinks packaged in 12 ounce portions generate the best sales.<br>
ii.Soft drinks is the top product based on sales value.


Below is the code and plots for the same:

In [ ]:
products_filter = products[products['department']=="GROCERY"]
product_transactions = products_filter.merge(transactions, on = ['product_id'], how = 'inner')
products_transactions_oz = product_transactions[product_transactions['package_size'].str.contains("OZ", na = False)]
products_transactions_lb = product_transactions[product_transactions['package_size'].str.contains("LB", na = False)]

products_transactions_oz_groupby = products_transactions_oz.groupby(['product_id','product_category','package_size'])\
                                            .aggregate({'sales_value':'sum'}).reset_index()\
                                            .sort_values('sales_value', ascending=False).head(10)

unidff = products_transactions_oz_groupby.groupby('product_category').first().reset_index().sort_values('sales_value', ascending=False)

unidff['package_size'] = unidff.package_size.str.extract('(\d+)')
unidff["package_size"] = pd.to_numeric(unidff["package_size"])
unidff = unidff.rename(columns = {'package_size':'package_size_in_ounces'})
# -----------------------------
products_transactions_lb_groupby = products_transactions_lb.groupby(['product_id','product_category','package_size'])\
                                            .aggregate({'sales_value':'sum'}).reset_index()\
                                            .sort_values('sales_value', ascending=False).head(7)

unidff_lb = products_transactions_lb_groupby.groupby('product_category').first().reset_index().sort_values('sales_value', ascending=False)

unidff_lb['package_size'] = unidff_lb.package_size.str.extract('(\d+)')
unidff_lb["package_size"] = pd.to_numeric(unidff_lb["package_size"])
unidff_lb = unidff_lb.rename(columns = {'package_size':'package_size_in_pounds'})

In [ ]:
labels = unidff.product_category
max_sales = unidff.sales_value
package_size_in_ounces = unidff.package_size_in_ounces

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
ax2 = ax.twinx()
rects1 = ax.bar(x - width/2, max_sales, width, label='max_sales', color = 'purple')
rects2 = ax2.bar(x + width/2, package_size_in_ounces, width, label='package_size_in_ounces', color = 'yellow')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Max sales')
ax2.set_ylabel('Package Size in ounces')
ax.set_title('Max sales and package size in ounces')
ax.set_xticks(x, labels)
ax.legend()
ax2.legend()

fig.autofmt_xdate(rotation=45)

plt.show()

In [ ]:
prod_chart = unidff.sort_values(by = ['sales_value'],
                    ascending = False).iloc[0:15].reset_index()


fig1 = go.Figure()
# Draw points
fig1.add_trace(go.Scatter(x = prod_chart["sales_value"], 
                          y = prod_chart["product_category"],
                          mode = 'markers',
                          marker_color ='darkblue',
                          marker_size  = 10))
# Draw lines
for i in range(0, len(prod_chart)):
               fig1.add_shape(type='line',
                              x0 = 0, y0 = i,
                              x1 = prod_chart["sales_value"][i],
                              y1 = i,
                              line=dict(color='crimson', width = 3))

fig1.layout.hovermode = False

fig1.update_layout(
    title_text="Top 5 product categories of Groceries having maximum sales<br><sup>This data has been collected over the year 2017</sup>"
)
fig1.show()

## Business Problem 3 
To analyse campaign performance based on coupons redemption rates, total sales generated by the products (which were part of the campaign), households targeted, ages and income ranges of the customers. 

- We utilized the completejourney dataset to identify trends and patterns in the data and generated insights to help the leadership make effective data-driven decisions.

- Throughout the analysis, we identify the best-case practices and patterns in certain campaigns and suggest areas of improvement to simulate the poorer performing campaigns.

- We analyzed the campaign performance based on the redemption rates, total sales generated by the products (which were part of the campaign), households targeted, ages and income ranges of the customers.

<b> Insights from the Graphs </b>

- Type A campaigns are performing better than the rest of the campaigns.
- Households with a family income between 50-74K are more likely to redeem the coupons followed by households with income range between 35-49K.
- Age group between the period of 45-54 has the highest number of campaign promotional redemption users followed by the age group 35-44.


In [ ]:
campaign_descriptions.head()

In [ ]:
coupon_redemptions.head()

In [ ]:
campaigns.head()

In [ ]:
coupons.head()

In [ ]:
# Compute campaign products total sales value for different campaign types

camp_prod_tsv = coupons[['campaign_id', 'product_id']] \
    .merge(campaign_descriptions[['campaign_id', 'campaign_type']], on='campaign_id') \
    .merge(transactions[['product_id', 'transaction_timestamp', 'sales_value']], on='product_id') \
    .groupby('campaign_type') \
    .agg(total_sales=('sales_value', 'sum')) \
    .reset_index() \
    .sort_values('campaign_type')

camp_prod_tsv

In [ ]:
# Households Targeted by campaigns 

camp_hh_targeted = campaigns.groupby('campaign_id', as_index=False) \
    .agg(hh_targeted=('household_id', 'nunique')) \
    .sort_values('campaign_id') \
    .merge(campaign_descriptions, on='campaign_id') \
    .loc[:, ['campaign_type', 'campaign_id', 'hh_targeted']] \


camp_hh_targeted_type = camp_hh_targeted.groupby('campaign_type', as_index=False) \
    .agg(total_hh_targeted =('hh_targeted', 'sum')) 

camp_hh_targeted_type

In [ ]:
# Coupons redemption by households

camp_hh_redeemed = campaigns[['campaign_id']] \
    .drop_duplicates() \
    .merge(coupon_redemptions[['campaign_id', 'household_id']].drop_duplicates(), on='campaign_id', how='left') \
    .groupby('campaign_id', as_index=False) \
    .agg(hh_redeemed=('household_id', 'nunique')) \
    .sort_values('campaign_id') \
    .merge(campaign_descriptions[['campaign_id', 'campaign_type']], on='campaign_id') \
    .assign(campaign_id=lambda x: pd.to_numeric(x['campaign_id'])) \
    .loc[:, ['campaign_id', 'campaign_type', 'hh_redeemed']] 


camp_hh_redeemed_type = camp_hh_redeemed.groupby('campaign_type', as_index=False) \
    .agg(total_hh_redeemed =('hh_redeemed', 'sum')) 

camp_hh_redeemed_type

In [ ]:
# Compute Campaign performance 

camp_perf_df = camp_hh_targeted.merge(camp_hh_redeemed, on='campaign_id', how='inner') \
    .sort_values('campaign_id') \
    .assign(redemption_rate=lambda x: (x['hh_redeemed']/x['hh_targeted'])*100) \
    .rename(columns={'campaign_type_x': 'campaign_type'}) \
    .loc[:, ['campaign_id', 'campaign_type', 'hh_targeted', 'hh_redeemed', 'redemption_rate']] \
    .sort_values(by='redemption_rate', ascending=False)

camp_perf_df

In [ ]:
# Compute max campaign ID
max_camp_df = campaigns[['campaign_id']].astype({'campaign_id': 'int'}).agg({'campaign_id': 'max'}).reset_index()
maximum_campaign_id = max_camp_df[0][0]
maximum_campaign_id

In [ ]:
# Campaign performance vs campaign type scatter plot

sns.set(rc={'figure.figsize':(11.7,8.27)})
plot= sns.catplot(data=camp_perf_df, x="campaign_id", y="redemption_rate", col = 'campaign_type', hue= 'campaign_type')
plot.set_axis_labels("campaign_id", "redemption_rate")
plot.fig.subplots_adjust(top=0.8)
plot.fig.suptitle("Overall Campaign Performance")

for ax in plot.axes.flat:
    labels = ax.get_xticklabels() # get x labels
    for i,l in enumerate(labels):
        if(i%4 != 0): labels[i] = '' # make every 4th label visible
    ax.set_xticklabels(labels) # set new labels
plt.show()
sns.despine(left=True)

In [ ]:
# Coupon Redemption Of Household – By Income Range

demo_income = pd.merge(demographics, coupon_redemptions, on='household_id', how='left')
demo_income['redemption'] = demo_income['redemption_date'].apply(lambda x: 'Never Used a promotion' if pd.isna(x) else 'Used a Promotion')
demo_income = demo_income[['age', 'income', 'redemption']]
demo_income = demo_income.groupby(['income', 'redemption']).size().reset_index(name='hh_count')

demo_income

In [ ]:
# Plot Households Redeeming Promotions by Income Range

demo_income_plot = sns.barplot(x='hh_count', y='income', hue='redemption', data=demo_income, palette=['#E69F00', '#56B4E9'])
demo_income_plot.set(title='Households Redeeming Promotions by Income Range', xlabel='Number of Households', ylabel='Income Range')
demo_income_plot.legend(title='Coupon Redemption?')
demo_income_plot.figure.subplots_adjust(left=0.3)
demo_income_plot.figure.set_size_inches(8, 6)
demo_income_plot.set_axisbelow(True)

In [ ]:
# Coupon Redemptions Of Household – By Age

demo_age = pd.merge(demographics, coupon_redemptions, on='household_id', how='left')
demo_age['redemption'] = demo_age['redemption_date'].apply(lambda x: 'Never Used a promotion' if pd.isna(x) else 'Used a Promotion')
demo_age = demo_age[['age', 'income', 'redemption']]
demo_age = demo_age.groupby(['age', 'redemption']).size().reset_index(name='hh_count')

demo_age

In [ ]:
# Plot Households Redeeming Promotions by Age

plt.figure(figsize=(8,6))
sns.barplot(x='hh_count', y='age', hue='redemption', data=demo_age, orient='horizontal', alpha=1)
plt.xlabel('Number of Households')
plt.ylabel('Age Group')
plt.title('Number of Households Redeeming Promotions by Age')
plt.legend(title='Coupon Redemption?')
plt.show()

<b> Summary </b>

- Target 45-54 age group while giving discounts since its yielding higher sales.
- Drinking water and soft drinks packaged in 12 ounce portions to be sold more as they generate the best sales.
- Type A campaigns are performing better than the rest of the campaigns.
- To attain Maximum redemption Rate, we should run the campaign on Households with a family income of 100k and above.
- Young adults in the age group of 19-24 and older citizens above age 55 have been least targeted by campaigns and have least promotional benefit users. We cam launch personalized campaigns for these age groups to boost sales.

<b> Limitations </b>

- Coupon related sales information is not reflected in the transaction dataset. Hence the analysis was made for the total sales of the products targeted by the campaigns.
- We have transaction data only for 2017 whereas campaign data is present from 2016 - 2018. Hence the analysis was limited to 2017. There is no granular information about the characteristics and other attributes about campaigns and coupon.
- There were too many scales of package sizes. We have carried out our analysis towards pounds and ounces.

<b> Future Scope </b>
- In our analysis we have taken discount versus age group. Similar analysis can be done on other Demographic information.
- We could further analyse Product v/s Discount analysis in future which could further generate more insights